# Wczytywanie danych z SQL

In [20]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('database.sqlite')

# Sprawdźmy, jakie tabele są dostępne w bazie danych
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
available_tables = cursor.fetchall()

print("Dostępne tabele w bazie danych:")
for table in available_tables:
    print(f"- {table[0]}")

sales_df = pd.read_sql("SELECT * FROM Sales", conn)
customers_df = pd.read_sql("SELECT * FROM Customers", conn)
cities_df = pd.read_sql("SELECT * FROM Cities", conn)
item_master_df = pd.read_sql("SELECT * FROM Item_Master", conn)
sales_rep_df = pd.read_sql("SELECT * FROM Sales_Rep", conn)
kursy = pd.read_sql("SELECT * FROM kursy_walut", conn)



Dostępne tabele w bazie danych:
- Cities
- Customers
- Item_Master
- Sales_Rep
- Sales
- kursy_walut


# Łączenie tabel

In [21]:
# 1. Łączenie Sales z Customers
sales_data = pd.merge(sales_df, customers_df, on='Customer Number', how='left')

# 2. Dołączanie Cities
print("Dołączanie Cities...")
sales_data = pd.merge(sales_data, cities_df, on='City Code', how='left')

# 3. Dołączanie Item_Master
print("Dołączanie Item_Master...")
sales_data = pd.merge(sales_data, item_master_df, on='Item Number', how='left')

print("Utworzono sales_data.")
print(f"Rozmiar końcowej tabeli: {sales_data.shape}")
print("Podgląd połączonych danych:")
print(sales_data.head())

Dołączanie Cities...
Dołączanie Item_Master...
Utworzono sales_data.
Rozmiar końcowej tabeli: (457837, 27)
Podgląd połączonych danych:
   %KEY    Cost  Customer Number                 Date  GrossSales  \
0  3428 -513.15         10012226  2019-01-12 00:00:00     -573.38   
1  3429 -105.93         10012226  2019-01-12 00:00:00     -204.66   
2  3430  -88.07         10012226  2019-01-12 00:00:00     -165.80   
3  3431  -43.12         10012226  2019-01-12 00:00:00     -118.37   
4  3432  -37.98         10012226  2019-01-12 00:00:00     -102.33   

          Invoice Date  Invoice Number                   Item Desc  \
0  2019-01-12 00:00:00          318960     Cutting Edge Sliced Ham   
1  2019-01-12 00:00:00          318960  Washington Cranberry Juice   
2  2019-01-12 00:00:00          318960             Moms Sliced Ham   
3  2019-01-12 00:00:00          318960                 Tip Top Lox   
4  2019-01-12 00:00:00          318960        Just Right Beef Soup   

   Item Number  Margin  ...  

#Agregacja sprzedaży

In [22]:
# łączna sprzedaż dla każdego regionu
# rupujemy po region ( doszedł z tabeli Cities) i sumujemy kolumnę sales
region_sales = sales_data.groupby('Region')['Sales'].sum().sort_values(ascending=False)


print("\nŁączna sprzedaż w regionach:")

print(region_sales)


Łączna sprzedaż w regionach:
Region
USA       213,215,607.70
UK        131,186,657.40
Japan      56,410,523.04
Nordic     51,453,707.98
Spain      16,137,566.70
Germany    15,924,492.16
Name: Sales, dtype: float64


In [23]:
print(sales_rep_df.head(3))

        Manager  Manager Number                         Path  Sales Rep Name  \
0  Amanda Honda             104    Amanda Honda-Amalia Craig    Amalia Craig   
1  Amanda Honda             104      Amanda Honda-Cart Lynch      Cart Lynch   
2  Amanda Honda             104  Amanda Honda-Molly McKenzie  Molly McKenzie   

  Sales Rep Name1 Sales Rep Name2 Sales Rep Name3  Sales Rep ID  
0    Amanda Honda    Amalia Craig            None           103  
1    Amanda Honda      Cart Lynch            None           112  
2    Amanda Honda  Molly McKenzie            None           159  


In [24]:
print(sales_df.head(3))

   %KEY    Cost  Customer Number                 Date  GrossSales  \
0  3428 -513.15         10012226  2019-01-12 00:00:00     -573.38   
1  3429 -105.93         10012226  2019-01-12 00:00:00     -204.66   
2  3430  -88.07         10012226  2019-01-12 00:00:00     -165.80   

          Invoice Date  Invoice Number                   Item Desc  \
0  2019-01-12 00:00:00          318960     Cutting Edge Sliced Ham   
1  2019-01-12 00:00:00          318960  Washington Cranberry Juice   
2  2019-01-12 00:00:00          318960             Moms Sliced Ham   

   Item Number  Margin  Order Number Promised Delivery Date   Sales  \
0        10696  -37.29        115785    2019-01-12 00:00:00 -550.44   
1        10009  -90.54        115785    2019-01-12 00:00:00 -196.47   
2        10385  -71.10        115785    2019-01-12 00:00:00 -159.17   

   Sales Qty  Sales Rep Number Currency  
0      -1.00               180      AUD  
1      -2.00               180      EUR  
2      -3.00               180 

In [25]:
# średnia marża
query_task_3_b = """
SELECT
    sr."Sales Rep Name",
    AVG(s.Margin) as "Średnia Marża"
FROM Sales s
JOIN Sales_Rep sr
    ON s."Sales Rep Number" = sr."Sales Rep ID"
GROUP BY
    sr."Sales Rep Name"
ORDER BY
    "Średnia Marża" DESC;
"""

srednia = pd.read_sql(query_task_3_b, conn)

print("\n--- Średnia marża dla każdego sprzedawcy ---")
pd.options.display.float_format = '{:,.2f}'.format
print(srednia)


--- Średnia marża dla każdego sprzedawcy ---
     Sales Rep Name  Średnia Marża
0          Lee Chin      18,709.61
1      Judy Thurman      11,117.41
2      Stewart Wind       4,421.05
3   Cheryle Sincock       2,082.13
4    Martha Richard       1,447.71
..              ...            ...
58       Cary Frank         105.43
59   Samantha Allen          98.53
60     Teresa Lynch          94.58
61      Lisa Taylor          90.46
62       Craig Lary          77.87

[63 rows x 2 columns]


# Filtrowanie danych

In [26]:
query_task_4 = """
SELECT *
FROM Sales s
LEFT JOIN Sales_Rep sr
    ON s."Sales Rep Number" = sr."Sales Rep ID"  -- Twoje łączenie
WHERE
    s.Sales > 1000
    AND s.Margin < 0
ORDER BY
    s.Margin ASC
LIMIT 10;
"""
top_10_sql = pd.read_sql(query_task_4, conn)

print("\n--- ZADANIE 4: Top 10 transakcji (Sales > 1000, Margin < 0) ---")
columns_to_show = ['Sales', 'Margin', 'Sales Rep ID']
print(top_10_sql[columns_to_show])


--- ZADANIE 4: Top 10 transakcji (Sales > 1000, Margin < 0) ---
     Sales    Margin  Sales Rep ID
0 2,177.28 -3,764.40           141
1 2,177.28 -3,764.40           141
2 2,177.28 -3,764.40           141
3 2,177.28 -3,764.40           141
4 2,268.00 -3,673.68           141
5 2,268.00 -3,673.68           141
6 2,268.00 -3,673.68           141
7 2,268.00 -3,673.68           141
8 2,268.00 -3,673.68           141
9 2,268.00 -3,673.68           141


# Grupy produktowe

In [27]:
# używamy ramki danych z kroku 2 (sales data)
product_group_stats = sales_data.groupby('Product Group')[['Sales', 'Sales Qty']].sum().reset_index()

# sortowanie i wybór Top 5
top_5_product_groups = product_group_stats.sort_values(by='Sales', ascending=False).head(5)
print("\n--- Top 5 grup produktowych z najwyższą sprzedażą ---")
print(top_5_product_groups)


--- Top 5 grup produktowych z najwyższą sprzedażą ---
      Product Group          Sales    Sales Qty
11          Produce 111,405,547.81 1,951,190.00
5   Canned Products  91,071,529.01 2,672,814.00
7              Deli  68,724,239.20   686,456.00
9      Frozen Foods  44,649,652.08   425,480.00
13           Snacks  39,754,586.67   436,887.00


# Podział plików sprzedaży na waluty

In [28]:
import os

folder_name = 'dane_walutowe'
os.makedirs(folder_name, exist_ok=True)

lista_walut = sales_df['Currency'].unique()

print(f"Znalezione waluty do przetworzenia: {lista_walut}")

for waluta in lista_walut:

    dane_dla_waluty = sales_df[sales_df['Currency'] == waluta]

    nazwa = waluta.replace('/', '_')
    nazwa_pliku = f"waluta_{nazwa}.csv"

    # sklejamy folder z nazwą pliku -> wynik np.: "dane_walutowe/waluta_EUR.csv"
    sciezka_zapisu = os.path.join(folder_name, nazwa_pliku)

    # index=False sprawia, że nie zapisujemy numerów wierszy (0, 1, 2...)
    dane_dla_waluty.to_csv(sciezka_zapisu, index=False)

    print(f" -> Zapisano plik: {nazwa_pliku}")

print("Wszystkie pliki zostały utworzone.")

Znalezione waluty do przetworzenia: ['AUD' 'EUR' 'USD' 'GBP' 'CHF' 'CAD']
 -> Zapisano plik: waluta_AUD.csv
 -> Zapisano plik: waluta_EUR.csv
 -> Zapisano plik: waluta_USD.csv
 -> Zapisano plik: waluta_GBP.csv
 -> Zapisano plik: waluta_CHF.csv
 -> Zapisano plik: waluta_CAD.csv
Wszystkie pliki zostały utworzone.


# Transakcje sprzedawców


In [29]:
query_task_7 = """
SELECT
    sr."Sales Rep Name",
    COUNT(s."Order Number") as "Order Number"
FROM Sales s
LEFT JOIN Sales_Rep sr
    ON s."Sales Rep Number" = sr."Sales Rep ID"
GROUP BY
    sr."Sales Rep ID",
    sr."Sales Rep Name"
ORDER BY
    "Order Number" DESC
LIMIT 3;
"""
top_3_sellers_sql = pd.read_sql(query_task_7, conn)

print("\n--- Top 3 Sprzedawców (SQL) ---")
print(top_3_sellers_sql)


--- Top 3 Sprzedawców (SQL) ---
  Sales Rep Name  Order Number
0    Brad Taylor         37720
1      TAGnology         28466
2  Karl Anderson         21590


# Wartość sprzedaży w walutach

In [30]:
sales_with_cur = pd.merge(sales_data, kursy, left_on='Currency', right_on='code', how='left')
# przeliczenie sprzedaży na PLN
sales_with_cur['Sales_PLN'] = sales_with_cur['Sales'] * sales_with_cur['mid']

# łączna sprzedaż w PLN dla regionu i waluty
region_currency_stats = sales_with_cur.groupby(['Region', 'Currency'])['Sales_PLN'].sum().reset_index()

region_currency_stats = region_currency_stats.sort_values(by=['Region', 'Sales_PLN'], ascending=[True, False])

print("\n--- Łączna sprzedaż w PLN wg regionu i waluty ---")
print(region_currency_stats)


--- Łączna sprzedaż w PLN wg regionu i waluty ---
     Region Currency      Sales_PLN
4   Germany      GBP  14,231,928.54
5   Germany      USD  10,655,863.29
2   Germany      CHF  10,240,487.90
1   Germany      CAD   8,762,595.04
0   Germany      AUD   8,395,629.63
3   Germany      EUR   8,262,100.16
10    Japan      GBP  49,532,449.08
11    Japan      USD  37,360,311.15
8     Japan      CHF  33,380,063.34
6     Japan      AUD  31,285,014.94
9     Japan      EUR  31,134,428.34
7     Japan      CAD  30,665,603.33
16   Nordic      GBP  48,176,554.91
14   Nordic      CHF  33,713,256.56
17   Nordic      USD  32,568,123.94
13   Nordic      CAD  28,064,726.09
15   Nordic      EUR  27,064,605.42
12   Nordic      AUD  26,858,293.88
22    Spain      GBP  13,275,728.57
20    Spain      CHF  11,494,535.90
23    Spain      USD   9,810,474.36
19    Spain      CAD   9,740,942.57
21    Spain      EUR   8,784,131.86
18    Spain      AUD   8,043,450.88
28       UK      GBP 107,012,012.57
29       UK  

# Analiza z wykorzystaniem apply

In [31]:

def klasyfikuj(margin):
    if margin < 0:
        return "Strata"
    elif margin == 0:
        return "Neutral"
    else:
        return "Zysk"

# sales_data['Margin'] to nasza kolumna z liczbami
# .apply(klasyfikuj) bierze każdą liczbę po kolei i przepuszcza przez funkcję
sales_data['Profit Category'] = sales_data['Margin'].apply(klasyfikuj)

print("\n--- Rozkład kategorii zysku ---")
print(sales_data['Profit Category'].value_counts())

print("\n--- Podgląd danych z nową kolumną ---")
print(sales_data[['Margin', 'Profit Category']].head(10))


--- Rozkład kategorii zysku ---
Profit Category
Zysk       438855
Strata      18844
Neutral       138
Name: count, dtype: int64

--- Podgląd danych z nową kolumną ---
   Margin Profit Category
0  -37.29          Strata
1  -90.54          Strata
2  -71.10          Strata
3  -70.52          Strata
4  -60.26          Strata
5  -32.79          Strata
6  -19.89          Strata
7  -52.12          Strata
8   -2.08          Strata
9    1.00            Zysk


# Tabela przestawna i eksport

In [32]:
# --- ZADANIE 10: Tabela przestawna i eksport ---

# 1. Tworzenie tabeli przestawnej
# values  -> co liczymy (Sprzedaż)
# index   -> co ma być w wierszach (Region)
# columns -> co ma być w kolumnach (Grupa produktowa)
# aggfunc -> funkcja, której używamy (suma)
pivot = sales_data.pivot_table(values='Sales',
                               index='Region',
                               columns='Product Group',
                               aggfunc='sum')

print("\n--- Tabela Przestawna (Region x Grupa Produktowa) ---")
print(pivot)

output_file = 'sprzedaz_regiony_produkty.xlsx'
pivot.to_excel(output_file)

print("Tabela zapisana do pliku: {output_file}")


--- Tabela Przestawna (Region x Grupa Produktowa) ---
Product Group  Alcoholic Beverages  Baked Goods  Baking Goods     Beverages  \
Region                                                                        
Germany                 175,853.63   176,290.48    625,397.71  1,034,551.06   
Japan                 5,724,265.60   419,288.71  2,505,992.79  3,381,942.34   
Nordic                  860,300.03   357,667.08  1,077,306.22  1,467,090.86   
Spain                   287,017.36   116,030.33    527,514.45  1,196,922.17   
UK                      730,977.04   486,163.08 19,702,480.96 10,243,352.10   
USA                   2,914,969.93   534,964.90  5,994,748.11 13,170,348.17   

Product Group  Breakfast Foods  Canned Products         Dairy          Deli  \
Region                                                                        
Germany              74,436.72     2,037,546.56  1,296,268.35  2,539,111.57   
Japan               182,716.49     6,414,767.11  6,604,613.56  9,494,626.98

#Propozycje podsumowań

## Ranking Top 5 miast z największą sprzedażą

In [33]:
query_task_11 = """
SELECT
    c.City,
    c.Region,
    COUNT(s."Order Number") as "Liczba Zamówień",
    SUM(s.Sales) as "Łączna Sprzedaż"
FROM Sales s
JOIN Customers cust ON s."Customer Number" = cust."Customer Number"
JOIN Cities c ON cust."City Code" = c."City Code"
GROUP BY
    c.City, c.Region
ORDER BY
    "Łączna Sprzedaż" DESC
LIMIT 5;
"""

top_cities_summary = pd.read_sql(query_task_11, conn)

print("\n--- Top 5 miast z największą sprzadażą ---")
print(top_cities_summary)


--- Top 5 miast z największą sprzadażą ---
        City Region  Liczba Zamówień  Łączna Sprzedaż
0    Cardiff     UK             3606    35,325,747.33
1  Nashville    USA             4876    28,464,238.72
2  Leicester     UK             8506    25,016,681.60
3     Denver    USA             3922    23,648,064.07
4  Baltimore    USA              823    19,808,067.72


## Kto generuje największy przychód

In [34]:
query_task_11_2 = """
SELECT
    sr."Sales Rep Name",
    SUM(s.Sales) as "Łączny Przychód",
    AVG(s.Margin) as "Średnia Marża"
FROM Sales s
JOIN Sales_Rep sr
    ON s."Sales Rep Number" = sr."Sales Rep ID"
GROUP BY
    sr."Sales Rep Name"
ORDER BY
    "Łączny Przychód" DESC
LIMIT 5;
"""

top_revenue_reps = pd.read_sql(query_task_11_2, conn)

print("\n--- Najlepsi Sprzedawcy (wg Przychodu) ---")
pd.options.display.float_format = '{:,.2f}'.format
print(top_revenue_reps)


--- Najlepsi Sprzedawcy (wg Przychodu) ---
  Sales Rep Name  Łączny Przychód  Średnia Marża
0   Judy Thurman    64,531,648.58      11,117.41
1   Stewart Wind    58,043,432.70       4,421.05
2       Lee Chin    41,959,884.90      18,709.61
3      John Greg    20,597,023.77         667.66
4    Brad Taylor    18,825,697.39         212.76
